In [ ]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.palettes import viridis, brewer, magma, d3, cividis
from bokeh.models import HoverTool, LinearColorMapper
from bokeh.models.tickers import FixedTicker
from bokeh.transform import linear_cmap, factor_cmap


## Read in and merge data

In [ ]:
df = pd.read_excel('../../data/raw/RVMS_Current_Property_and_BIZ_Owner_List - vCurrent (1).xlsx',
                  sheet_name = 'Biz & Prop Owner MAIN list')

naics = pd.read_excel('../../data/raw/2-6 digit_2017_Codes.xlsx')

In [ ]:
df['NAICS Code'] = df['NAICS Code'].astype(object)

In [ ]:
df = df.merge(naics, left_on='NAICS Code', right_on='2017 NAICS US   Code', how='inner')

In [ ]:
df = df[pd.notnull(df['NAICS Code'])]

In [ ]:
df.columns = [c.replace(' ', '_') for c in df.columns]

## Generate fake data for viz purposes

In [ ]:
n_rows = df.shape[0]
p = 0.25

cols = ['R2B_email_sponsorship_promotion', 'R2B_provide_resources', 'R2B_liason', 'B2R_event_participation',
       'B2R_sponsorship_donation', 'B2R_share_business_information', 'B2R_volunteer', 'B2R_use_RVMS_resources']

for col in cols:
    df[col] = np.random.binomial(n=1, p=p, size=n_rows)
    
df['R2B_score'] = df.filter(like='R2B').apply('sum',axis=1)
df['B2R_score'] = df.filter(like='B2R').apply('sum',axis=1)

df['R2B_score_noise'] = df['R2B_score'] + np.random.uniform(low=0, high = 0.5, size=n_rows)
df['B2R_score_noise'] = df['B2R_score'] + np.random.uniform(low=0, high = 0.5, size=n_rows)

## Visualization

In [ ]:
# Colors
cats = df['NAICS_Code'].astype(str).str[:2]
colors = viridis(len(cats.unique()))
colormap = {val: colors[i] for i, val in enumerate(cats.unique())}
df['color'] = [colormap[x] for x in cats]

# Radius
df['radius'] = 0.05

# Set up data source
source = ColumnDataSource(df)

# Hover tool
TOOLTIPS = [
    ("index", "$index"),
    ("Business", "@Business_Name"),
    ("Industry", "@2017_NAICS_US_Title"),
    ('Business to RVMS Score', "@B2R_score"),
    ('RVMS to Business Score', "@R2B_score"),
]

p = figure(title="Business Engagement Matrix", x_axis_label='Business to RVMS', y_axis_label='RVMS to Business',
          x_range=(0, 6), y_range=(0, 4), tooltips=TOOLTIPS)

# Ticks
p.xaxis.ticker = FixedTicker(ticks=[1, 2, 3, 4, 5])
p.yaxis.ticker = FixedTicker(ticks=[1, 2, 3, 4])

# Grid lines
p.xgrid.ticker = FixedTicker(ticks=[1,2,3,4,5])
p.ygrid.ticker = FixedTicker(ticks=[1,2,3,4])

p.scatter(x='B2R_score_noise', y='R2B_score_noise', fill_color='color', radius='radius', source=source)

In [ ]:
show(p)

In [ ]:
from bokeh.plotting import figure, output_file, save

In [ ]:
output_file("../../data/output/matrix.html")
save(p)